In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [13]:
from PortfolioToolboxQP.PortfolioToolboxQP.Qp_Solver import qp_solver_cpp as solve

Il faut remplir le code suivant avec ce qu'on a calculé dans les autres fichiers. Faut juste faire attention à ce qu'on est les mêmes colonnes pour tous sinon on va avoir des problèmes de dimensions.

In [48]:
sigma = np.load("data/cov_matrix_sigma.npy")
CI = pd.read_csv("data/filtered_CarbonIntensity.csv")
CI0 = CI["CI_Scope12_FY23"].values.reshape(-1, 1)
print(type(CI0))
print(type(sigma))

bench = (CI["Weight"].values / CI["Weight"].sum()).reshape(-1, 1)

print(bench.shape)
print(CI0.shape)
R = 0.35  # reduction_rate cf net_zero_emissions for other values

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(1040, 1)
(1040, 1)


J'ai pris le problème le plus simple avec le moins de contrainte (min 1/2(x-b)Tsigma(x-b) avec comme contraintes la carbone intensity, et le bail C0 des long only constraints [x entre 0 et 1 et qui se somme à 1])

In [53]:
y = solve(
    Q=sigma,
    p=None,
    G=np.diag(CI0[:,0]),
    h=(1 - R) * CI0*bench - CI0*bench,
    A= np.ones((bench.shape[0],1)).T,
    b=-np.ones(bench.shape).T @ bench + 1,
    lb= - bench,
    ub=np.ones(bench.shape) - bench,
)

""" x is what we want : here the relative weights on the differents firms. The following Y is what we call "x-b". 
 We have to add the benchmark at the end

                            minimize 0.5*Y.T@Q@Y + p@Y under constraints AY = B and GY <= h and  lb <= Y <= ub
    
    all parameters are ndarray

    Q: sigma for us
    p: 0

    CI*(X=Y+b) <= (1-R)CI_0*b donc CI*Y <= (1-R)*CI_0*b - CI*b
    G: Matrix of inequalty constraints → CI
    h: Vector of inequality constraints → -R*CI*b

    1.T(Y+b)=1 donc 1TY = - 1Tb + 1
    A: Matrix of equality constraints → 1T
    b: Vector of equality constraints → -1Tb + 1
    
    lb: Lower bound for Y : for x : 0
    ub: Upper bound for Y : for x : 1

    return: optimal vector if found : Y=X-b
    """

x = y + bench
print(x)
tracking_error = 0.5 * y.T @ sigma @ y
print(tracking_error)

c:\Users\vahye\Desktop\ENSAE\projet_finance\net-zero-portfolio\PortfolioToolboxQP\PortfolioToolboxQP\Qp_Solver.py:137: UserWarning: Warning : The p matrix in the QP problem was set to 0 as it was either empty or None -- Continuing optimization !
  warnings.warn(
c:\Users\vahye\Desktop\ENSAE\projet_finance\net-zero-portfolio\PortfolioToolboxQP\PortfolioToolboxQP\Qp_Solver.py:189: UserWarning: Warning : b was reshaped as its shape was not of form (n,)
  warnings.warn("Warning : b was reshaped as its shape was not of form (n,)")
c:\Users\vahye\Desktop\ENSAE\projet_finance\net-zero-portfolio\PortfolioToolboxQP\PortfolioToolboxQP\Qp_Solver.py:202: UserWarning: Warning : h was reshaped as its shape was not of form (n,)
  warnings.warn("Warning : h was reshaped as its shape was not of form (n,)")


[[ 6.81916142e-04  9.48997129e-04  9.81156463e-04 ... -4.28907588e-03
   8.31096169e-04  8.04096441e-04]
 [-8.11723952e-05  1.85908593e-04  2.18067926e-04 ... -5.05216442e-03
   6.80076322e-05  4.10079046e-05]
 [-1.73056205e-04  9.40247833e-05  1.26184117e-04 ... -5.14404823e-03
  -2.38761772e-05 -5.08759047e-05]
 ...
 [ 1.48847505e-02  1.51518315e-02  1.51839908e-02 ...  9.91375847e-03
   1.50339305e-02  1.50069308e-02]
 [ 2.55687492e-04  5.22768480e-04  5.54927813e-04 ... -4.71530453e-03
   4.04867519e-04  3.77867792e-04]
 [ 3.32829571e-04  5.99910558e-04  6.32069892e-04 ... -4.63816245e-03
   4.82009598e-04  4.55009870e-04]]
0.00019522722577655866
